# about the Dataset:

1. unnamed: unique id for a news article
2. title: the title of a news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:

      0: Fake news

      1: real News

# Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re          # regular expression used for searching text in a doc
from nltk.corpus import stopwords               # natural language tool kits
from nltk.stem.porter import PorterStemmer      # stemming : root word
from sklearn.feature_extraction.text import TfidfVectorizer   # to convert text into feature vectors(numbers)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')     # it is a collection of words which dont carry meaningful data of predictions ex: 'we' , 'us' , etc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Pre-processing

In [ ]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/drive/MyDrive/machine_learning_lab/news.csv')

In [ ]:
news_dataset.shape

(6335, 4)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()
# if there is any null value then we can replace it with empty strings by (   news_dataset = news_dataset.fillna('')  ) this fxn.

,0
Unnamed: 0,0
title,0
text,0
label,0


In [ ]:
news_dataset['content'] = news_dataset['title']

In [ ]:
print(news_dataset['content'])

0                            You Can Smell Hillary’s Fear
1       Watch The Exact Moment Paul Ryan Committed Pol...
2             Kerry to go to Paris in gesture of sympathy
3       Bernie supporters on Twitter erupt in anger ag...
4        The Battle of New York: Why This Primary Matters
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [ ]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)    # axis=1 represent whole colomn ans axis=0 represents whole row
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  \
0     Daniel Greenfield, a Shillman Journalism Fello...   
1     Google Pinterest Digg L

In [ ]:
# convert Y from string to int type ('FAKE' : 0 and 'REAL' : 1)
Y = Y.map({'FAKE': 0, 'REAL': 1})
print(Y)

0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Name: label, Length: 6335, dtype: int64


Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)   # only selecting text with alphabets from para and neglrcting int , special symbols from para and it will be replaced by space
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()          # arranging uneven spaced data into list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)            # after stemming joining all theewords
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0                                      smell hillari fear
1       watch exact moment paul ryan commit polit suic...
2                           kerri go pari gestur sympathi
3          berni support twitter erupt anger dnc tri warn
4                           battl new york primari matter
                              ...                        
6330       state depart say find email clinton specialist
6331                        p pb stand plutocrat pentagon
6332             anti trump protest tool oligarchi inform
6333    ethiopia obama seek progress peac secur east a...
6334                jeb bush suddenli attack trump matter
Name: content, Length: 6335, dtype: object


In [ ]:
X = news_dataset['content'].values

In [ ]:
print(X)

['smell hillari fear'
 'watch exact moment paul ryan commit polit suicid trump ralli video'
 'kerri go pari gestur sympathi' ...
 'anti trump protest tool oligarchi inform'
 'ethiopia obama seek progress peac secur east africa'
 'jeb bush suddenli attack trump matter']


In [ ]:
X.shape

(6335,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()   # term frequency, inverse document frequency (counts the importance of word by checking how much time it apears in text) and assign integer value to it
vectorizer.fit(X)                        # tf = no. of time a particular word occurs/total no. of words
                                         # idf = log(total no. of words/1+no. of time a particular word occurs)
X = vectorizer.transform(X)

In [ ]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 45882 stored elements and shape (6335, 6884)>
  Coords	Values
  (0, 2230)	0.5606827254461214
  (0, 2817)	0.31703295849951846
  (0, 5631)	0.7649346276717717
  (1, 1212)	0.3538286538491304
  (1, 2101)	0.44595444447238664
  (1, 3939)	0.3279355637571237
  (1, 4462)	0.26757273402295384
  (1, 4630)	0.24403179029804972
  (1, 4905)	0.29382045650093463
  (1, 5263)	0.29163264316148546
  (1, 5933)	0.3501812302002013
  (1, 6315)	0.13561903129801023
  (1, 6571)	0.21736822482437437
  (1, 6671)	0.27855534550881683
  (2, 2536)	0.5345737509664292
  (2, 2576)	0.3357697230253524
  (2, 3335)	0.41740393299177314
  (2, 4428)	0.37615749278439464
  (2, 6015)	0.5345737509664292
  (3, 215)	0.41916879208304547
  (3, 561)	0.2969312816565521
  (3, 1764)	0.3507454498930073
  (3, 2048)	0.4448394256850793
  (3, 5956)	0.27803614912134916
  (3, 6288)	0.3148734995784654
  :	:
  (6330, 5809)	0.3024865273895357
  (6331, 4474)	0.5790025183139886
  (6331, 4493)	0

In [ ]:
print(X.toarray())  #sparse matrix

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Splitting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)  # stratify:splitting test and train in equal proportion of 0,1 in Y
                      # random_state: to split dataset in same manner each time we run the code

# Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression() # sigmoid fxn

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

# accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data by Logistic Regression: ', training_data_accuracy)

Accuracy score of the training data by Logistic Regression:  0.9078531965272296


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data by Logistic Regression : ', test_data_accuracy)

Accuracy score of the test data by Logistic Regression :  0.8334648776637726


# Making a Predictive System

In [ ]:
X_new = X_test[0]       # 100 means 100th row in our test sample

prediction = model.predict(X_new)
print(prediction)

if (prediction==0):
  print('The news is fake')
else:
  print('The news is real')

[1]
The news is real


In [ ]:
print(Y_test.iloc[100])
# Instead of using a specific numerical index, use .iloc to access the element by its positional index.
# This will work regardless of the Series's index values.

1


In [ ]:
def predict_logistic_regression(news_article):

  # Preprocess the news article
  input_data = [news_article]
  input_data = vectorizer.transform(input_data)

  # Make prediction using the trained Logistic Regression model
  prediction = model.predict(input_data)

  return prediction[0]

# Example usage
news_article = "my name is pushpendra"
prediction = predict_logistic_regression(news_article)

if prediction == 0:
  print('The news is fake')
else:
  print('The news is real')

The news is fake


# Training the model: SVM

In [ ]:
from sklearn.svm import SVC

# Create an SVM classifier object
svm_classifier = SVC()

# Train the classifier
svm_classifier.fit(X_train, Y_train)

# Make predictions on the test set
svm_predictions = svm_classifier.predict(X_test)

# Evaluate the model's performance
svm_accuracy = accuracy_score(svm_predictions, Y_test)
print("SVM Accuracy on test dataset:", svm_accuracy)

SVM Accuracy on test dataset: 0.8413575374901342


In [ ]:
def predict_svm(news_article):

  # Preprocess the news article
  input_data = [news_article]
  input_data = vectorizer.transform(input_data)

  # Make prediction using the trained SVM model
  prediction = svm_classifier.predict(input_data)

  return prediction[0]

# Example usage
news_article = "my name is pushpendra"
prediction = predict_svm(news_article)

if prediction == 0:
  print('The news is fake')
else:
  print('The news is real')

The news is real


# Training the model: Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Create a Naive Bayes classifier object
nb_classifier = MultinomialNB()

# Train the classifier
nb_classifier.fit(X_train, Y_train)

# Make predictions on the test set
nb_predictions = nb_classifier.predict(X_test)

# Evaluate the model's performance
nb_accuracy = accuracy_score(nb_predictions, Y_test)
print("Naive Bayes Accuracy on test dataset:", nb_accuracy)

Naive Bayes Accuracy on test dataset: 0.8145224940805051


In [ ]:
def predict_naive_bayes(news_article):

  # Preprocess the news article
  input_data = [news_article]
  input_data = vectorizer.transform(input_data)

  # Make prediction using the trained Naive Bayes model
  prediction = nb_classifier.predict(input_data)

  return prediction[0]

# Example usage
news_article = "my name is pushpendra"
prediction = predict_naive_bayes(news_article)

if prediction == 0:
  print('The news is fake')
else:
  print('The news is real')

The news is real
